In [1]:
##imports from libraries
import pandas as pd
import numpy as np
import time
import math
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import resource
import os

In [2]:
## Preprocessing of data
# Load data here:
# get data function
def get_data(data_folder = '/home/yipingx/Data/ghg_data/'):
    data = []
    filelist = os.listdir(data_folder)
    for file in filelist:
#         print(file)
        data_single = np.genfromtxt(data_folder+file,dtype=np.float)
        data.append(data_single)
#     data=np.genfromtxt('./ghg_data/ghg.gid.site2067.dat',dtype=np.float)
    return data

data = np.array(get_data())

In [3]:
print(data.shape)

(2921, 16, 327)


In [3]:
# Split train and test data here: (X_train, Y_train, X_test, Y_test)


def splitDataset_v1(totaldata, train = 0.8, seed = 123, normalize = True):
    # seed
    np.random.seed(seed)
    # number
    numdata = totaldata.shape[0]
    numtrain = int(numdata*train)
    numtest = numdata - numtrain
    # index
    index = np.arange(numdata)
    np.random.shuffle(index)
    # shuffle
    totaldata = totaldata[index,:,:]
    # split
    traindata = totaldata[:numtrain, :, :]
    testdata = totaldata[numtrain:, :, :]
    # split X, Y
    # train
    X_train = traindata[:,:15, :]
    X_train = X_train.reshape(numtrain, -1).T
    Y_train = traindata[:,15, :].T
#     Y_train = Y_train.reshape(numtrain, 1, -1)
    
    # test
    X_test = testdata[:,:15, :]
    X_test = X_test.reshape(numtest, -1).T
    Y_test = testdata[:,15, :].T
    
    # normalization
    if normalize is True:
        X_train = X_train/np.linalg.norm(X_train, axis=0)
#         Y_train = Y_train/np.linalg.norm(Y_train, axis=0)
        X_test = X_test/np.linalg.norm(X_test, axis=0)
#         Y_test = Y_test/np.linalg.norm(Y_test, axis=0)
    
#     Y_test = Y_test.reshape(numtest, 1, -1)
    return X_train, Y_train, X_test, Y_test

def splitDataset_v2(totaldata, train = 0.8, seed = 123, normalize = True):
    # seed
    np.random.seed(seed)
    # number
    numdata = totaldata.shape[0]
    numtrain = int(numdata*train)
    numtest = numdata - numtrain
    # index
    index = np.arange(numdata)
    np.random.shuffle(index)
    # shuffle
    totaldata = totaldata[index,:,:]
    # split
    traindata = totaldata[:numtrain, :, :].reshape(numtrain,-1)
    testdata = totaldata[numtrain:, :, :].reshape(numtest,-1)
    # split X, Y
    # train
    X_train = traindata[:,:-1].T
    Y_train = traindata[:,-1].reshape(-1,1).T
    
    # test
    X_test = testdata[:,:-1].T
    Y_test = testdata[:,-1].reshape(-1,1).T
    
    # normalization
    if normalize is True:
        X_train = X_train/np.linalg.norm(X_train, axis=0)
#         Y_train = Y_train/np.linalg.norm(Y_train, axis=0)
        X_test = X_test/np.linalg.norm(X_test, axis=0)
#         Y_test = Y_test/np.linalg.norm(Y_test, axis=0)
    
#     Y_test = Y_test.reshape(numtest, 1, -1)
    return X_train, Y_train, X_test, Y_test
# Split train and test data here: (X_train, Y_train, X_test, Y_test)
X_train, Y_train, X_test, Y_test = splitDataset_v2(data)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(5231, 2336)
(1, 2336)
(5231, 585)
(1, 585)


In [5]:
D, N = X_train.shape
d,_ = Y_train.shape
i=1
w = np.random.normal(0,0.1, D*d).reshape(D,d)

In [6]:
w.shape

(5231, 1)

In [4]:
Z = Y_train[:,i] * np.dot(w.T , (X_train[:,i].reshape(D,1)).reshape(D,1)) # 1 x 1
dw = np.zeros((D, d))
dw = (X_train[:,i].reshape(D,1) * Y_train[:,i]/(1 + np.exp(Z))).reshape(-1)
print(Z)
print(dw[:].shape)
print(X_train[:,i].reshape(D,1).shape)
print((Y_train[:,i]/(1 + np.exp(Z))).shape)

NameError: name 'i' is not defined

In [5]:
## Logistic ridge regression with different optimizers
# cost function and gradient calculation
def cost_vectorization(x,y,w,lambda_=0.01):
    # x.shape (5231, 2336)
    # y.shape (1, 2336)
    # w.shape (5231, 1)
    D, N = x.shape
    value = 0
    Z = -1 * y * np.dot(w.T , x) # (1, 2336)
    value = np.sum(np.log(1+np.exp(Z)))
    norm_w = np.linalg.norm(w)
    c = lambda_ * norm_w ** 2
    return value/N + c


def cost(x,y,w,lambda_ = 0.01):
    D, N = x.shape
    value = 0
    for i in range(N):
        Z = -1 * y[:,i] * np.dot(w.T , (x[:,i].reshape(D,1)).reshape(D,1))
        value += np.sum(np.log(1+np.exp(Z))) # y 1-d value
#         value += np.sum(np.log(1+np.exp(Z))) / y.shape[0] # y 327-d vector
    norm_w = np.linalg.norm(w)
    c = lambda_ * norm_w ** 2
#     print("=========================")
#     print("loss part 1: {}".format(value/N))
#     print("loss part 2, lambda term : {}".format(c))
    return value/N + c 



def function_gradient_vectorization(X, Y, w, lambda_, gradclip = None):
    # x.shape (5231, 2336)
    # y.shape (1, 2336)
    # w.shape (5231, 1)
    # dw.shape (5231,1)
    D, N = X.shape
    d, _ = Y.shape
    
#     dw = np.zeros((D, d))
    dw = np.zeros((D, d))
    Z = Y * np.dot(w.T , X) # (1, 2336)
    dw = np.dot(X , (Y/(1 + np.exp(Z))).T)
    # dw = X * Y/(1 + np.exp(Z))

    c = lambda_ * w *2
    
    if gradclip != None:
        c[c>=gradclip] = gradclip
    
    return c-dw/N



def function_gradient(X, Y, w, lambda_, gradclip = None):
    # Calculate the gradient here:
    # X: DxN
    # Y: dxN
    # w: Dxd
    D, N = X.shape
    d, _ = Y.shape
    
#     dw = np.zeros((D, d))
    dw = np.zeros((D, d))
    
    for i in range(N):
#         for j in range(d):
#             Z = Y[j,i] * np.dot(w[:,j].reshape(1,-1) , (X[:,i].reshape(D,1)).reshape(D,1)) # 1 x 1
            
#             dw[:,j] += (X[:,i].reshape(D,1) * Y[j,i]/(1 + np.exp(Z))).reshape(-1) # D x 1        
        Z = Y[:,i] * np.dot(w.T , X[:,i].reshape(D,1)) # 1 x 1
#         up = X[:,i].reshape(D,1) * Y[:,i]
#         down = (1 + np.exp(Z[0]))
        dw[:] += X[:,i].reshape(D,1) * Y[:,i]/(1 + np.exp(Z[0]))
#         dw[:] += (X[:,i].reshape(D,1) * Y[:,i]/(1 + np.exp(Z[0]))).reshape(-1) # D x 1
    c = lambda_ * w *2
    
    if gradclip != None:
        c[c>=gradclip] = gradclip
    
    return c-dw/N

In [6]:
y = Y_train
x = X_train
D,N = x.shape
d,_ = y.shape
lambda_ = 0.0001
# w = np.random.rand(D,1)*0.01  # Initialization of w
w = np.random.normal(0,0.1, D*d).reshape(D,d)
L = cost(x,y,w,lambda_)
print(L)

print(x.shape)
print(y.shape)
print(w.shape)
g = function_gradient(x, y, w, lambda_)
print(g.shape)

7.46468415545774
(5231, 2336)
(1, 2336)
(5231, 1)
(5231, 1)


In [38]:
print(cost_vectorization(x,y,w,lambda_))

g1 = function_gradient_vectorization(x, y, w, lambda_)
print(g1.shape)

0.5979731355935242
(5231, 1)


In [41]:
diff = g-g1
rel_diff = np.abs(diff)/np.abs(g)
print(rel_diff.max())
print(np.abs(diff).max())

3.4832110676678154e-15
3.3306690738754696e-16


In [66]:
## Define solvers: GD, SGD, SVRG and SAG. 
# Setting the values here:

alpha = 0.1 # change the value
num_iters = 5 # change the value
lambda_ = 0.0001 # change the value
epsilon = 0.001 # change the value

# ---------------------- Complete the blank definitions: --------------------------------------

def solver(x,y, w, alpha, num_iters , lambda_ , epsilon , optimizer = "GD", batchsize = None, gradclip = None, mem=False):
    if (optimizer == "GD") :
#         print(num_iters)
        for i in range(num_iters):
            # update the parameter w for GD here:
            g = function_gradient_vectorization(x, y, w, lambda_, gradclip)
            w = w - alpha*g
#             loss = cost(x,y,w,lambda_)
#             print(i, loss)
            if (i%500 == 0):
                loss = cost_vectorization(x,y,w,lambda_)
                print(i, loss)
            if (i%10==0) and (mem):
                usage=resource.getrusage(resource.RUSAGE_SELF)
                print("mem for GD (mb):", (usage[2]*resource.getpagesize())/1000000.0)
            if (np.linalg.norm(g) <= epsilon):
                break
    elif (optimizer == "SGD"):
        for i in range(num_iters):
            # Complete SGD here:
                # randomly choose NSample points for calculating the estimated gradient
                if (batchsize == None):
                    NSample = int(np.random.rand() * x.shape[1])
                    while NSample == 0:
                        NSample = int(np.random.rand() * x.shape[1])
                else:
                    NSample = batchsize
                randomInd = np.arange(x.shape[1])[:NSample]
                g = function_gradient_vectorization(x[:,randomInd], y[:,randomInd], w, lambda_, gradclip = gradclip)
                w = w - alpha*g
                
                if (i%500 == 0):
                    loss = cost_vectorization(x,y,w,lambda_)
                    print(i, loss)
                
                if (i%10==0) and (mem):
                    usage=resource.getrusage(resource.RUSAGE_SELF)
                    print("mem for GD (mb):", (usage[2]*resource.getpagesize())/1000000.0)
                if (np.linalg.norm(g) <= epsilon):
                    break
                
                
    elif (optimizer == "SVRG"):
        i = 0
        T = 2000
#         K = math.floor(num_iters/T)
        K = 100
#         Z = np.matmul(x,np.diagflat(y))
        N = x.shape[1]
        for k in range(K):
#             wz = np.matmul(w.T , Z)
#             diag = np.diagflat(1/(1+np.exp(-1*wz))-np.ones((1,N)))
#             Ga_ = np.matmul(Z , diag)
#             ga_ = (1/N) * np.matmul(Ga_ , np.ones((N,1)))
            # compute all gradient and store
            g = function_gradient_vectorization(x, y, w, lambda_, gradclip)
            # initialize the w_previous
            w_previous = w.copy()
            for t in range(T):
                # random sample
                if (batchsize == None):
                    NSample = int(np.random.rand() * x.shape[1])
                    while NSample == 0:
                        NSample = int(np.random.rand() * x.shape[1])
                else:
                    NSample = batchsize
                # randomInd = np.arange(x.shape[1])[:NSample]
                randomInd = np.arange(x.shape[1])
                np.random.shuffle(randomInd)
                randomInd = randomInd[:NSample]

                if t==0 and k==0:
                    print(randomInd)
                # calculate the update term
#                 w_previous = w.copy()
                part1 = function_gradient_vectorization(x[:,randomInd], y[:,randomInd], w_previous, lambda_, gradclip = gradclip)
                part2 = function_gradient_vectorization(x[:,randomInd], y[:,randomInd], w, lambda_, gradclip = gradclip)
                part3 = g

                w_previous = w_previous - alpha * (part1 - part2 + part3)
                
                i = i+1
                if (i % 1000 == 0):
                    loss = cost_vectorization(x,y,w_previous,lambda_)
                    print(i, loss)
            w = w_previous
            if (np.linalg.norm((part1 - part2 + part3)) <= epsilon):
                break
                
    elif (optimizer == "SAG"):
        # Complete SAG here:
        # X: DxN
        # Y: dxN
        # w: Dxd
        D, N = x.shape
        d = y.shape[0]
        dw = np.zeros_like(w, dtype=np.float)
        gk = np.zeros((N,D,d), dtype=np.float)


        for k in range(num_iters):
            index = np.random.randint(0,N)
            # NSample = int(np.random.rand() * x.shape[1])
            NSample  = batchsize

            # randomInd = np.arange(x.shape[1])[:NSample]
            randomInd = np.arange(x.shape[1])
            np.random.shuffle(randomInd)
            randomInd = randomInd[:NSample]

            index_all = randomInd
            # if k==0:
                # print(index)
            # for index in index_all:
            g_i = function_gradient_vectorization(x[:,index].reshape(D,1), y[:,index].reshape(d,1), w, lambda_, gradclip = gradclip)
            # dw = dw - gk[index,:,:] +  g_i

            gk[index,:,:] = g_i
            # dw =  ( dw + 2 * lambda_ * w)/N
            # n = min(k+1,N)
            # dw = dw /N
            dw = np.sum(gk[index,:,:],axis=0)
            w = w - alpha * dw/N
            if (np.linalg.norm(dw) <= epsilon):
                break

            if (k % 100 == 0):
                    loss = cost_vectorization(x,y,w,lambda_)
                    # print(dw[:20,:])
                    print(k, loss)
            if (k%10==0) and (mem):
                usage=resource.getrusage(resource.RUSAGE_SELF)
                print("mem for SAG (mb):", (usage[2]*resource.getpagesize())/1000000.0)
            # i_t = 
        i=k
    return w,i

In [67]:
## Solving the optimization problem:

alpha = 0.01 # change the value
num_iters = 50000 # change the value
lambda_ = 0.01 # change the value
epsilon = 0.001 # change the value

y = Y_train
x = X_train
D,N = x.shape
d,_ = y.shape

# w = np.random.rand(D,1)*0.01  # Initialization of w
# np.random.seed(1000)
w = np.random.normal(0,0.1, D*d).reshape(D,d)
# start = time.time()
#-------------------- GD Solver -----------------------
loss = cost_vectorization(x,y,w,lambda_)
print("initial loss {}".format(loss))

#-------------------- GD Solver -----------------------
# print("GD optimization")
# opt = "GD"
# start_gd = time.time()
# gde, iters_total = solver(x,y, w, alpha, num_iters  , lambda_ , epsilon , optimizer = opt, mem=False) # complete the command 
# end_gd = time.time()
# cost_value = cost_vectorization(x,y,gde,lambda_)  # Calculate the cost value
# print("Cost of GD after convergence: ",cost_value)
# print("Training time for GD: ", end_gd-start_gd)

# # #-------------------- SGD Solver -----------------------
# # complete here :
# opt = 'SGD'
# print("SGD optimization")
# start_sgd = time.time()
# gde, iters_total = solver(x,y, w, alpha, num_iters  , lambda_ , epsilon , optimizer = opt, mem=False) # complete the command 
# end_sgd = time.time()
# cost_value = cost_vectorization(x,y,gde,lambda_)  # Calculate the cost value
# print("Cost of SGD after convergence: ",cost_value)
# print("Training time for SGD: ", end_sgd-start_sgd)

# #-------------------- SVRG Solver -----------------------
# # complete here :
# alpha = 0.1 # change the value
# num_iters = 2000 # change the value
# lambda_ = 0.0001 # change the value
# epsilon = 0.0000001 # change the value
# opt = 'SVRG'
# print("SVRG optimization")
# start_srvg = time.time()
# gde, iters_total = solver(x,y, w, alpha, num_iters  , lambda_ , epsilon , optimizer = opt, mem=False, batchsize = 50) # complete the command 
# end_svrg = time.time()
# cost_value = cost_vectorization(x,y,gde,lambda_)  # Calculate the cost value
# print("Cost of SVRG after convergence: ",cost_value)
# print("Training time for SVRG: ", end_svrg-start_srvg)


#-------------------- SAG Solver -----------------------
# complete here :
alpha = 0.01 # change the value
num_iters = 20000 # change the value
lambda_ = 0.0001 # change the value
epsilon = 0.0000001 # change the value
opt = 'SAG'
print("SAG optimization")
start_sag = time.time()
gde, iters_total = solver(x,y, w, alpha, num_iters, lambda_ , epsilon , optimizer = opt, mem=False, batchsize = 50) # complete the command 
end_sag = time.time()
cost_value = cost_vectorization(x,y,gde,lambda_)  # Calculate the cost value
print("Cost of SAG after convergence: ",cost_value)
print("Training time for SAG: ", end_sag-start_sag)

initial loss 2.1006450449168645
SAG optimization
0 0.04689019605784502
100 0.03973485541641171
200 0.03559048679294316
300 0.033648385856915944
400 0.03223616512210999
500 0.02962528218785076
600 0.028562887762184835
700 0.02766316929068601
800 0.027010200317596263
900 0.026430186418533952
1000 0.025835770235811236
1100 0.025187386774815156
1200 0.024744251826225385
1300 0.024253615010602424
1400 0.02392007311331483
1500 0.023613023528042787
1600 0.02336055114072531
1700 0.023106008286414993
1800 0.02277578738498129
1900 0.022400036537156394
2000 0.022103534921787342
2100 0.02182367984103336
2200 0.021711942267968496
2300 0.021563186968844062
2400 0.021365476663004347
2500 0.021191175061577784
2600 0.02107931521279479
2700 0.020761895733869065
2800 0.020684472996815743
2900 0.02052807481068481
3000 0.02023556937808976
3100 0.020062817459377904
3200 0.019805018720074848
3300 0.01956865508578081
3400 0.019339044703369765
3500 0.019214181213433216
3600 0.019111139631100057
3700 0.01896048

In [ ]:
''## Executing the iterations and plot the cost function here:

ti= np.zeros((50,4))
cost_= np.zeros((50,4))
for i in range(50):
    print("......",i,".......")
    #--------------GD-------------------
    start = time.time()
    gde = solver(x.T,y,w,num_iters=i)
    end = time.time()

    cost_[i,0] = cost(x.T,y,gde)

    ti[i,0] = end-start

    #---------------SGD------------------
    #Complete for SGD solver here :
    
    #---------------SVRG----------------
    #Complete for SVRG solver here :
    
    #---------------SAG------------------
    #Complete for SAG solver here :
    
    #------------------------------------
    
    ## Pl the results:
    

l0 = plt.plot(cost_[:,0],color="red")
# complete other plots here: 


plt.xlabel("Number of Iteration")
plt.ylabel("Cost")
plt.legend(['GD', 'SGD', 'SVRG', 'SAG'])
plt.show()

l0 = plt.plot(ti[:,0],color="red")
# complete other plots here:

plt.xlabel("Number of Iteration")
plt.ylabel("Time (sec)")
plt.legend(['GD', 'SGD', 'SVRG', 'SAG'])
plt.show()
    
    
    

In [ ]:
## Tunning the hyper-paramter here:

In [ ]:
## Comparing different optimizers here: 